# 7장 WaveNet: 심층 학습을 기반으로 음성 파형 생성 모델

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch07_WaveNet.ipynb)

## 준비

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

### 패키지 임포트

In [ ]:
%pylab inline
%load_ext autoreload
%load_ext tensorboard
%autoreload
import IPython
from IPython.display import Audio
import tensorboard as tb
import os

In [ ]:
# 수치 연산
import numpy as np
import torch
from torch import nn
# 음성 파형 불러오기
from scipy.io import wavfile
# 사운드 분석 및 시각화
import librosa
import librosa.display
# 파이썬에서 배우는 음성 합성
import ttslearn

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(773)

In [ ]:
torch.__version__

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

## 7.3 WaveNet에서 음성 파형 처리

### $\mu$-law 알고리즘

In [ ]:
def mulaw(x, mu=255):
    return np.sign(x) * np.log1p(mu * np.abs(x)) / np.log1p(mu)

def quantize(y, mu=255, offset=1):
    # [-1, 1] -> [0, 2] -> [0, 1] -> [0, mu]
    return ((y + offset) / 2 * mu).astype(np.int64)    

def mulaw_quantize(x, mu=255):
    return quantize(mulaw(x, mu), mu)

#### $\mu$-law 알고리즘 적용 전

In [ ]:
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768.0).astype(np.float32)

mu = 2**8-1 # 8-bit

fig, ax = plt.subplots(2, 1, figsize=(6,4))
ax[0].set_title("Waveform")
ax[1].set_title("Histrogram")

ax[0].set_ylim(-0.9, 0.9)
librosa.display.waveshow(x, ax=ax[0], sr=16000)

ax[1].set_xlim(-0.9, 0.9)
ax[1].hist(x, bins=mu)

ax[0].set_xlabel("Time [sec]")
ax[0].set_ylabel("Amplitude")
ax[1].set_xlabel("Amplitude")
ax[1].set_ylabel("Count")

plt.tight_layout()

# 그림 7-6 (a)
savefig("./fig/wavenet_mulaw_a")

#### $\mu$-law 알고리즘 적용 후

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6,4))
ax[0].set_title("Waveform")
ax[1].set_title("Histrogram")

ax[0].set_ylim(-0.9, 0.9)
librosa.display.waveshow(mulaw(x), ax=ax[0], sr=16000)

ax[1].set_xlim(-0.9, 0.9)
ax[1].hist(mulaw(x), bins=mu)

ax[0].set_xlabel("Time [sec]")
ax[0].set_ylabel("Amplitude")
ax[1].set_xlabel("Amplitude")
ax[1].set_ylabel("Count")

plt.tight_layout()

# 그림 7-6 (b)
savefig("./fig/wavenet_mulaw_b")

### $\mu$-law 알고리즘에 의한 역변환

In [ ]:
def inv_mulaw(y, mu=255):
    return np.sign(y) * (1.0 / mu) * ((1.0 + mu)**np.abs(y) - 1.0)

def inv_quantize(y, mu):
    # [0, mu] -> [-1, 1]
    return 2 * y.astype(np.float32) / mu - 1

def inv_mulaw_quantize(y, mu=255):
    return inv_mulaw(inv_quantize(y, mu), mu)

#### $\mu$-law 없음

In [ ]:
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768.0).astype(np.float32)  
x = librosa.resample(x, sr, 16000)
sr = 16000

bits = [8, 4]

fig, ax = plt.subplots(len(bits)+1, 1, figsize=(6,2*(len(bits)+1)), sharey=True)
ax[0].set_title("Input waveform")
librosa.display.waveshow(x, sr, x_axis="time", ax=ax[0])
IPython.display.display(Audio(x, rate=sr))

for idx, bit in enumerate(bits):
    mu = 2**bit - 1
    x_hat = inv_quantize(quantize(x, mu), mu)
    librosa.display.waveshow(x_hat, sr, x_axis="time", ax=ax[idx+1])
    ax[idx+1].set_title(f"{bit}-bit waveform")
    IPython.display.display(Audio(x_hat, rate=sr))

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
    a.set_xticks(np.arange(0, 3.5, 0.5))
    a.set_ylim(-0.5, 0.5)
plt.tight_layout()

# 그림 7-7 (a)
savefig("./fig/wavenet_inv_mulaw_waveform_a")

#### $\mu$-law 있음

In [ ]:
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768.0).astype(np.float32)  
x = librosa.resample(x, sr, 16000)
sr = 16000

bits = [8, 4]

fig, ax = plt.subplots(len(bits)+1, 1, figsize=(6,2*(len(bits)+1)), sharey=True)
ax[0].set_title("Input waveform")
librosa.display.waveshow(x, sr, x_axis="time", ax=ax[0])
IPython.display.display(Audio(x, rate=sr))

for idx, bit in enumerate(bits):
    mu = 2**bit - 1
    x_hat = inv_mulaw_quantize(mulaw_quantize(x, mu), mu)
    librosa.display.waveshow(x_hat, sr, x_axis="time", ax=ax[idx+1])
    ax[idx+1].set_title(f"{bit}-bit waveform")
    IPython.display.display(Audio(x_hat, rate=sr))

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
    a.set_xticks(np.arange(0, 3.5, 0.5))
    a.set_ylim(-0.5, 0.5)
plt.tight_layout()

# 그림 7-7 (b)
savefig("./fig/wavenet_inv_mulaw_waveform_b")

## 7.4 인과적인 팽창 컨벌루션 (Dilated Convolution)

### 1차원 컨벌루션

In [ ]:
def _toy_1d_input():
    # (B, C, T) where B and C = 1
    return torch.tensor([1,2,3,0,1,2,4],dtype=torch.float).view(1,1,-1)

#### 패딩을 하지 않는 경우

In [ ]:
conv = nn.Conv1d(1,1,3,bias=False, padding=0)
conv.weight.data[0,0,:] = torch.tensor([1,2,4],dtype=torch.float)

x = _toy_1d_input()
with torch.no_grad():
    y= conv(x)
print("입력:", x.long().view(-1).tolist())
print("출력:", y.long().view(-1).tolist())

#### 패딩을 할 때

In [ ]:
conv = nn.Conv1d(1,1,3,bias=False, padding=1)
conv.weight.data[0,0,:] = torch.tensor([1,2,4],dtype=torch.float)

x = _toy_1d_input()
with torch.no_grad():
    y= conv(x)
print("입력:", x.long().view(-1).tolist())
print("출력:", y.long().view(-1).tolist())

#### 2층의 1차원 컨벌루션

In [ ]:
conv = nn.Conv1d(1,1,3,bias=False, padding=1)
conv.weight.data[0,0,:] = torch.tensor([1,2,4],dtype=torch.float)

x = _toy_1d_input()
with torch.no_grad():
    y= conv(conv(x))
print("입력:", x.long().view(-1).tolist())
print("출력:", y.long().view(-1).tolist())

### 인과적인 컨벌루션 (Dilated Convolution)

In [ ]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()
        self.padding = (kernel_size - 1)
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=self.padding, **kwargs)

    def forward(self, x):
        # 1차원 컨벌루션
        y = self.conv(x)
        # 인과성을 담보하기 위해 순방향(forward)으로 이동
        if self.padding > 0:
            y = y[:, :, :-self.padding]
        return y

In [ ]:
conv = CausalConv1d(1,1,3,bias=False)
# 테스트를 위해 컨벌루션 커널을 수동으로 설정
conv.conv.weight.data[0,0,:] = torch.tensor([1,2,4],dtype=torch.float)

x = _toy_1d_input()
y= conv(x)
print("입력:", x.long().view(-1).tolist())
print("출력:", y.long().view(-1).tolist())

### 1차원 팽창 컨벌루션 (Dilated Convolution)

In [ ]:
class DilatedCausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, **kwargs):
        super().__init__()
        # 패딩의 너비를 계산할 때 dilation factor를 고려해야합니다.
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=self.padding, dilation=dilation, **kwargs)

    def forward(self, x):
        # 1차원 컨벌루션
        y = self.conv(x)
        # 인과성을 담보하기 위해 순방향으로 이동
        if self.padding > 0:
            y = y[:, :, :-self.padding]
        return y

In [ ]:
conv = DilatedCausalConv1d(1,1,3,dilation=2, bias=False)
# 테스트를 위해 컨벌루션 커널을 수동으로 설정
conv.conv.weight.data[0,0,:] = torch.tensor([1,2,4],dtype=torch.float)

x = _toy_1d_input()
y= conv(x)
print("입력:", x.long().view(-1).tolist())
print("출력:", y.long().view(-1).tolist())

## 7.5 게이트화된 활성화 함수를 이용한 1 차원 컨볼 루션

In [ ]:
class GatedDilatedCausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels*2, kernel_size, padding=self.padding, dilation=dilation)
        
    def forward(self, x):
        # 1차원 컨벌루션
        y = self.conv(x)
        
        # 인과성을 담보하기 위해 순방향으로 이동
        if self.padding > 0:
            y = y[:, :, :-self.padding]

        # 채널 방향으로 분할
        a, b = y.split(y.size(1) // 2, dim=1)
        
        # 게이트화된 활성화 함수 적용
        y = torch.tanh(a) * torch.sigmoid(b)

        return y

In [ ]:
conv = GatedDilatedCausalConv1d(128, 16, 3, dilation=2)
x = torch.ones(32, 128, 100)
print("입력 사이즈:", tuple(x.shape))
print("출력 사이즈:", tuple(conv(x).shape))

## 7.6 조건부 특징 량의 업 샘플링

### 반복 기반 업샘플링

In [ ]:
x = torch.tensor([[1, 2, 3],[1, 2, 3],[1,2,3]]).view(1,3,-1).float()
y = nn.Upsample(scale_factor=3, mode="nearest")(x)
print(x)
print(y)

In [ ]:
class RepeatUpsampling(nn.Module):
    def __init__(self, upsample_scales):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=np.prod(upsample_scales), mode="nearest")

    def forward(self, c):
        return self.upsample(c)

In [ ]:
c = torch.ones(32, 80, 10)
# 예를 들어 100배로 업샘플링
c_up = RepeatUpsampling([100])(c)

print("입력 사이즈:", tuple(c.shape))
print("출력 사이즈:", tuple(c_up.shape))

### 최근접 보간(nearest neighbor interpolation)과 컨벌루션의 병용에 기초한 업샘플링

In [ ]:
from torch.nn import functional as F

class UpsampleNetwork(nn.Module):
    def __init__(self, upsample_scales):
        super().__init__()
        self.upsample_scales = upsample_scales
        self.conv_layers = nn.ModuleList()
        for scale in upsample_scales:
            kernel_size = (1, scale * 2 + 1)
            conv = nn.Conv2d(
                1, 1, kernel_size=kernel_size, padding=(0, scale), bias=False
            )
            conv.weight.data.fill_(1.0 / np.prod(kernel_size))
            self.conv_layers.append(conv)

    def forward(self, c):
        # (B, 1, C, T)
        c = c.unsqueeze(1)
        # 최근접 보간과 컨벌루션 반복
        for idx, scale in enumerate(self.upsample_scales):
            # 시간 방향으로만 업샘플링
            # (B, 1, C, T) -> (B, 1, C, T*scale)
            c = F.interpolate(c, scale_factor=(1, scale), mode="nearest")
            c = self.conv_layers[idx](c)
        # B x C x T
        return c.squeeze(1)

In [ ]:
c = torch.ones(32, 80, 10)
c_up = UpsampleNetwork([10, 8])(c)

print("입력 사이즈:", tuple(c.shape))
print("출력 사이즈:", tuple(c_up.shape))

#### 실제 데이터(mel-spectrogram) 업샘플링(bonus)

책에서는 해설하지 않았지만, 2차원 컨벌루션의 가중치를 적절히 초기화하는 것으로, 컨벌루션 전후에 스케일이 유지되는 것을 나타냅니다.

In [ ]:
# 초기화의 영향을 확인하기 위해, 컨벌루션 파라미터를 난수로 초기화
class RandomInitUpsampleNetwork(UpsampleNetwork):
    def __init__(self, upsample_scales):
        super().__init__(upsample_scales)
        for conv in self.conv_layers:
            nn.init.normal_(conv.weight.data, 0, 1.0)

In [ ]:
from ttslearn.dsp import logmelspectrogram

_sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768.0).astype(np.float32)
sr = 16000
x = librosa.resample(x, _sr, sr)
hop_length = int(0.0125 * sr)
sp = logmelspectrogram(x, sr, hop_length=hop_length)

fig, ax = plt.subplots(figsize=(8,4))
mesh = librosa.display.specshow(sp.T, sr=sr, hop_length=hop_length, cmap=cmap, x_axis="time", y_axis="frames")
fig.colorbar(mesh, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Frequency [Hz]")
plt.tight_layout()

Audio(x, rate=sr)

In [ ]:
upsample_net = UpsampleNetwork([10, 8])
upsample_net

In [ ]:
tsp = torch.from_numpy(sp.T).view(1, 80, -1)

# 컨벌루션 커널을 적절히 초기화한 경우
tsp_up = upsample_net(tsp)

# 무작위로 초기화한 경우
torch.manual_seed(0)
upsample_net_rand_init = RandomInitUpsampleNetwork([10, 8])

tsp_up_rand_init = upsample_net_rand_init(tsp)

A = tsp.squeeze(0).numpy()
B = tsp_up_rand_init.squeeze(0).detach().numpy()
C = tsp_up.squeeze(0).detach().numpy()

s, e = 100, 120

fig, ax = plt.subplots(1, 3, figsize=(10,5))
ax[0].set_title("Mel-spectrogram")
ax[1].set_title("Upsample (random init)")
ax[2].set_title("Upsample (proper init)")

ax[0].set_xlim(s, e)
ax[0].imshow(A, aspect="auto", interpolation="nearest", origin="lower", cmap=cmap)
fig.colorbar(ax[0].pcolormesh(A, cmap=cmap, rasterized=True), ax=ax[0])

ax[1].set_xlim(s*80, e*80)
ax[1].imshow(B, aspect="auto", interpolation="nearest", origin="lower", cmap=cmap)
fig.colorbar(ax[1].pcolormesh(B, cmap=cmap, rasterized=True), ax=ax[1])

ax[2].set_xlim(s*80, e*80)
ax[2].imshow(C, aspect="auto", interpolation="nearest", origin="lower", cmap=cmap)
fig.colorbar(ax[2].pcolormesh(C, cmap=cmap, rasterized=True), ax=ax[2])

for a in ax:
    # 나중에 다시 라벨을 붙이기 때문에 여기에서 지워 둡니다.
    a.set_ylabel("")

ax[0].set_ylabel("Mel filter channel")
ax[0].set_xlabel("Time [frame]")
for a in ax[1:]:
    a.set_xlabel("Time [sample]")
    
plt.tight_layout()

### 주변의 조건부 특징량을 고려한 업 샘플링

In [ ]:
class ConvInUpsampleNetwork(nn.Module):
    def __init__(self, upsample_scales, cin_channels, aux_context_window):
        super(ConvInUpsampleNetwork, self).__init__()
        # 조건부 특징량의 시간 방향으로 인접한 정보를 1차원 컨벌루션으로 고려
        kernel_size = 2 * aux_context_window + 1
        self.conv_in = nn.Conv1d(cin_channels, cin_channels, kernel_size, bias=False)
        # 업샘플링
        self.upsample = UpsampleNetwork(upsample_scales)

    def forward(self, c):
        c_up = self.upsample(self.conv_in(c))
        return c_up

In [ ]:
c = torch.ones(32, 80, 10)

c_up = ConvInUpsampleNetwork([10, 8], 80, 2)(c)
print("입력 사이즈:", tuple(c.shape))
print("출력 사이즈:", tuple(c_up.shape))

## 7.7 WaveNet 구현

### 1 x 1 컨벌루션

In [ ]:
def Conv1d1x1(in_channels, out_channels, bias=True):
    return nn.Conv1d(
        in_channels, out_channels, kernel_size=1, padding=0, dilation=1, bias=bias
    )


### 컨벌루션 블록

In [ ]:
class ResSkipBlock(nn.Module):
    def __init__(
        self,
        residual_channels,  # 잔차 결합 채널 (residual channel) 수
        gate_channels,  # 게이트 채널 수
        kernel_size,  # 커널 크기
        skip_out_channels,  # 건너뛰기 조인 채널 수
        dilation=1,  # dilation factor
        cin_channels=80,  # 조건부 특징 량의 채널 수
        *args,
        **kwargs,
    ):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation

        # 1차원 팽창 컨벌루션(dilation == 1일 때는 일반 1차원 컨벌루션)
        self.conv = nn.Conv1d(
            residual_channels,
            gate_channels,
            kernel_size,
            padding=self.padding,
            dilation=dilation,
            *args,
            **kwargs,
        )

        # local conditioning 용 1x1 convolution
        self.conv1x1c = Conv1d1x1(cin_channels, gate_channels, bias=False)

        # 게이트 된 활성화 함수로 인해 1 차원 컨벌루션 출력은 2 분할됩니다.
        gate_out_channels = gate_channels // 2
        self.conv1x1_out = Conv1d1x1(gate_out_channels, residual_channels)
        self.conv1x1_skip = Conv1d1x1(gate_out_channels, skip_out_channels)

    def forward(self, x, c):
        # 잔차 연결을 위한 입력 유지
        residual = x

        # 1차원 컨벌루션
        splitdim = 1  # (B, C, T)
        x = self.conv(x)
        # 인과성을 보장하기 위해 출력을 이동
        x = x[:, :, : -self.padding]

        # 채널 방향으로 출력 분할
        a, b = x.split(x.size(1) // 2, dim=1)

        # local conditioning
        c = self.conv1x1c(c)
        ca, cb = c.split(c.size(1) // 2, dim=1)
        a, b = a + ca, b + cb

        # 게이트화된 활성화 함수
        x = torch.tanh(a) * torch.sigmoid(b)

        # 건너뛰기 연결을 위한 출력 계산
        s = self.conv1x1_skip(x)

        # 잔차 연결의 요소 합을하기 전에 차원 수를 맞춥니다.
        x = self.conv1x1_out(x)

        x = x + residual

        return x, s

In [ ]:
kernel_size = 3
conv = ResSkipBlock(128,16,kernel_size, 64, dilation=4)
x = torch.ones(32, 128, 100)
c = torch.ones(32, 80, 100)
out, skip = conv(x, c)
out.shape, skip.shape

### WaveNet 전체 구현

In [ ]:
# 수용 필드의 크기를 수식대로 단순하게 계산
(2 - 1) * sum([1,2,4,8,16,32,64,128,256,512]) * 3 + 1

In [ ]:
# 수용 필드의 크기를 계산하는 함수
from ttslearn.wavenet import receptive_field_size

for layers, stacks, kernel_size in [
    (30, 3, 2), # WaveNet 논문 설정
]:
    print(f"[Layers: {layers}, Dilation cycles: {stacks}, kernel size: {kernel_size}]: recepive field (ミリ秒):")
    size = receptive_field_size(layers, stacks, kernel_size)
    print(f"{size} samples ({size / 16000 * 1000} ミリ秒)")

In [ ]:
class WaveNet(nn.Module):
    def __init__(
        self,
        out_channels=256,  # 출력 채널 수
        layers=30,  # 레이어 수
        stacks=3,  # 컨벌루션 블록 수
        residual_channels=64,  # 잔차 결합 채널 수
        gate_channels=128,  # 게이트 채널 수
        skip_out_channels=64,  # 건너뛰기 연결 채널 수
        kernel_size=2,  # 1차원 컨벌루션 커널 크기
        cin_channels=80,  # 조건부 특징 량의 채널 수
        upsample_scales=None,  # 업샘플링 스케일
        aux_context_window=0,  # 업샘플링 시 참조하는 이웃 프레임 수
    ):
        super().__init__()
        self.out_channels = out_channels
        self.cin_channels = cin_channels
        self.aux_context_window = aux_context_window
        if upsample_scales is None:
            upsample_scales = [10, 8]
        self.upsample_scales = upsample_scales

        self.first_conv = Conv1d1x1(out_channels, residual_channels)

        # 메인이 되는 컨벌루션층
        self.main_conv_layers = nn.ModuleList()
        layers_per_stack = layers // stacks
        for layer in range(layers):
            dilation = 2 ** (layer % layers_per_stack)
            conv = ResSkipBlock(
                residual_channels,
                gate_channels,
                kernel_size,
                skip_out_channels,
                dilation=dilation,
                cin_channels=cin_channels,
            )
            self.main_conv_layers.append(conv)

        # 건너뛰기 연결의 합을 파형으로 변환
        self.last_conv_layers = nn.ModuleList(
            [
                nn.ReLU(),
                Conv1d1x1(skip_out_channels, skip_out_channels),
                nn.ReLU(),
                Conv1d1x1(skip_out_channels, out_channels),
            ]
        )

        # 프레임 단위의 특징량을 샘플 단위로 업샘플링
        self.upsample_net = ConvInUpsampleNetwork(
            upsample_scales, cin_channels, aux_context_window
        )

    def forward(self, x, c):
        # 양자화 된 이산수열을 One-hot 벡터로 변환
        # (B, T) -> (B, T, out_channels) -> (B, out_channels, T)
        x = F.one_hot(x, self.out_channels).transpose(1, 2).float()

        # 조건부 특징 량의 업 샘플링
        c = self.upsample_net(c)

        # One-hot 벡터의 차원을 숨겨진 레이어의 차원으로 변환
        x = self.first_conv(x)

        # 메인 컨벌루션 레이어 처리
        # 각 레이어에서 스킵 연결의 출력을 더하여 유지
        skips = 0
        for f in self.main_conv_layers:
            x, h = f(x, c)
            skips += h

        # 건너 뛰기 연결의 합계를 입력으로 사용하여 출력 계산
        x = skips
        for f in self.last_conv_layers:
            x = f(x)

        # NOTE: 출력을 확률 값으로 해석하는 경우 softmax가 필요하지만,
        # 학습시에는 nn.CrossEntropyLoss의 계산에 두고 softmax의 계산이 행해지므로,
        # 여기서는 명시 적으로 softmax를 계산할 필요가 없습니다.
        return x

### 장난감 모델을 이용한 WaveNet의 동작 확인

In [ ]:
# NOTE: inference와 호환되는 WaveNet을 사용하려면 다음 줄을 주석 처리하십시오.
# from ttslearn.wavenet import WaveNet

# 여기서는 inference 함수의 구현을 생략합니다.

wavenet = WaveNet(out_channels=256, layers=2, stacks=1, kernel_size=2, cin_channels=64)
wavenet

In [ ]:
# 0에서 255 사이의 값을 갖는 적절한 입력 신호
x = torch.randint(0, 255, (16, 16000))
# 프레임 시프트를 80개 샘플로 하여 64차원 조건부 특징량 생성
c = torch.rand(16, 64, 16000//80)

print("입력 사이즈:", tuple(x.shape))
print("조건부 특징량의 크기:", tuple(c.shape))

x_hat = wavenet(x, c)

# 업샘플링 동작 확인을 위해 조건부 특징량의 업샘플링만 수행
c_up = wavenet.upsample_net(c)

print("업샘플링된 조건부 특징량의 크기:", tuple(c_up.shape))
print("WaveNet 출력 크기:", tuple(x_hat.shape))

### 음의 로그 우도(likelihood) 최소화 구현

In [ ]:
log_prob = F.log_softmax(x_hat, dim=1)
# 자기 회귀성을 유지하기 위해 출력을 시간 방향으로 하나 이동
nll = nn.NLLLoss()(log_prob[:, :, :-1], x[:, 1:])

In [ ]:
ce_loss = nn.CrossEntropyLoss()(x_hat[:, :, :-1], x[:, 1:])
print("nll:", nll.item())
print("ce_loss", ce_loss.item())